Attack *Algorithm*

In [7]:
## l2_attack.py -- attack a network optimizing for l_2 distance
##
## Copyright (C) 2016, Nicholas Carlini <nicholas@carlini.com>.
##
## This program is licenced under the BSD 2-Clause licence,
## contained in the LICENCE file in this directory.

import sys
import tensorflow as tf
import numpy as np


###After testing the boolean variable, targeted == false, Glenn determined that the score was more accurate when it was not targeted. 
### Then, Glenn moved on to manipulate the different integer and float variables using hyperparameter tuning




BINARY_SEARCH_STEPS = 9  # number of times to adjust the constant with binary search
MAX_ITERATIONS = 10000   # number of iterations to perform gradient descent
ABORT_EARLY = True       # if we stop improving, abort gradient descent early
LEARNING_RATE = 1e-2     # larger values converge faster to less accurate results
TARGETED = False          # should we target one specific class? or just be wrong?
CONFIDENCE = 0           # how strong the adversarial example should be
INITIAL_CONST = 1e-3     # the initial constant c to pick as a first guess

### GLENN's changed
###Assessing model holistically, assessing model's variables

### Will be using for loop to see if these are indeed the best variables for the job

class CarliniL2:
    def __init__(self, sess, model, batch_size=1, confidence = CONFIDENCE,
                 targeted = TARGETED, learning_rate = LEARNING_RATE,
                 binary_search_steps = BINARY_SEARCH_STEPS, max_iterations = MAX_ITERATIONS,
                 abort_early = ABORT_EARLY, 
                 initial_const = INITIAL_CONST,
                 boxmin = -0.5, boxmax = 0.5):
        """
        The L_2 optimized attack. 

        This attack is the most efficient and should be used as the primary 
        attack to evaluate potential defenses.

        Returns adversarial examples for the supplied model.

        confidence: Confidence of adversarial examples: higher produces examples
          that are farther away, but more strongly classified as adversarial.
        batch_size: Number of attacks to run simultaneously.
        targeted: True if we should perform a targetted attack, False otherwise.
        learning_rate: The learning rate for the attack algorithm. Smaller values
          produce better results but are slower to converge.
        binary_search_steps: The number of times we perform binary search to
          find the optimal tradeoff-constant between distance and confidence. 
        max_iterations: The maximum number of iterations. Larger values are more
          accurate; setting too small will require a large learning rate and will
          produce poor results.
        abort_early: If true, allows early aborts if gradient descent gets stuck.
        initial_const: The initial tradeoff-constant to use to tune the relative
          importance of distance and confidence. If binary_search_steps is large,
          the initial constant is not important.
        boxmin: Minimum pixel value (default -0.5).
        boxmax: Maximum pixel value (default 0.5).
        """

        image_size, num_channels, num_labels = model.image_size, model.num_channels, model.num_labels
        self.sess = sess
        self.TARGETED = targeted
        self.LEARNING_RATE = learning_rate
        self.MAX_ITERATIONS = max_iterations
        self.BINARY_SEARCH_STEPS = binary_search_steps
        self.ABORT_EARLY = abort_early
        self.CONFIDENCE = confidence
        self.initial_const = initial_const
        self.batch_size = batch_size

        self.repeat = binary_search_steps >= 10

        self.I_KNOW_WHAT_I_AM_DOING_AND_WANT_TO_OVERRIDE_THE_PRESOFTMAX_CHECK = False

        shape = (batch_size,image_size,image_size,num_channels)
        
        # the variable we're going to optimize over
        modifier = tf.Variable(np.zeros(shape,dtype=np.float32))

        # these are variables to be more efficient in sending data to tf
        self.timg = tf.Variable(np.zeros(shape), dtype=tf.float32)
        self.tlab = tf.Variable(np.zeros((batch_size,num_labels)), dtype=tf.float32)
        self.const = tf.Variable(np.zeros(batch_size), dtype=tf.float32)

        # and here's what we use to assign them
        self.assign_timg = tf.placeholder(tf.float32, shape)
        self.assign_tlab = tf.placeholder(tf.float32, (batch_size,num_labels))
        self.assign_const = tf.placeholder(tf.float32, [batch_size])
        
        # the resulting image, tanh'd to keep bounded from boxmin to boxmax
        self.boxmul = (boxmax - boxmin) / 2.
        self.boxplus = (boxmin + boxmax) / 2.
        self.newimg = tf.tanh(modifier + self.timg) * self.boxmul + self.boxplus
        
        # prediction BEFORE-SOFTMAX of the model
        self.output = model.predict(self.newimg)
        
        # distance to the input data
        self.l2dist = tf.reduce_sum(tf.square(self.newimg-(tf.tanh(self.timg) * self.boxmul + self.boxplus)),[1,2,3])
        
        # compute the probability of the label class versus the maximum other
        real = tf.reduce_sum((self.tlab)*self.output,1)
        other = tf.reduce_max((1-self.tlab)*self.output - (self.tlab*10000),1)

        if self.TARGETED:
            # if targetted, optimize for making the other class most likely
            loss1 = tf.maximum(0.0, other-real+self.CONFIDENCE)
        else:
            # if untargeted, optimize for making this class least likely.
            loss1 = tf.maximum(0.0, real-other+self.CONFIDENCE)

        # sum up the losses
        self.loss2 = tf.reduce_sum(self.l2dist)
        self.loss1 = tf.reduce_sum(self.const*loss1)
        self.loss = self.loss1+self.loss2
        
        # Setup the adam optimizer and keep track of variables we're creating
        start_vars = set(x.name for x in tf.global_variables())
        optimizer = tf.train.AdamOptimizer(self.LEARNING_RATE)
        self.train = optimizer.minimize(self.loss, var_list=[modifier])
        end_vars = tf.global_variables()
        new_vars = [x for x in end_vars if x.name not in start_vars]

        # these are the variables to initialize when we run
        self.setup = []
        self.setup.append(self.timg.assign(self.assign_timg))
        self.setup.append(self.tlab.assign(self.assign_tlab))
        self.setup.append(self.const.assign(self.assign_const))
        
        self.init = tf.variables_initializer(var_list=[modifier]+new_vars)

    def attack(self, imgs, targets):
        """
        Perform the L_2 attack on the given images for the given targets.

        If self.targeted is true, then the targets represents the target labels.
        If self.targeted is false, then targets are the original class labels.
        """
        r = []
        print('go up to',len(imgs))
        for i in range(0,len(imgs),self.batch_size):
            print('tick',i)
            r.extend(self.attack_batch(imgs[i:i+self.batch_size], targets[i:i+self.batch_size]))
        return np.array(r)

    def attack_batch(self, imgs, labs):
        """
        Run the attack on a batch of images and labels.
        """
        def compare(x,y):
            if not isinstance(x, (float, int, np.int64)):
                x = np.copy(x)
                if self.TARGETED:
                    x[y] -= self.CONFIDENCE
                else:
                    x[y] += self.CONFIDENCE
                x = np.argmax(x)
            if self.TARGETED:
                return x == y
            else:
                return x != y

        batch_size = self.batch_size

        # convert to tanh-space
        imgs = np.arctanh((imgs - self.boxplus) / self.boxmul * 0.999999)

        # set the lower and upper bounds accordingly
        lower_bound = np.zeros(batch_size)
        CONST = np.ones(batch_size)*self.initial_const
        upper_bound = np.ones(batch_size)*1e10

        # the best l2, score, and image attack
        o_bestl2 = [1e10]*batch_size
        o_bestscore = [-1]*batch_size
        o_bestattack = [np.zeros(imgs[0].shape)]*batch_size
        
        for outer_step in range(self.BINARY_SEARCH_STEPS):
            print(o_bestl2)
            # completely reset adam's internal state.
            self.sess.run(self.init)
            batch = imgs[:batch_size]
            batchlab = labs[:batch_size]
    
            bestl2 = [1e10]*batch_size
            bestscore = [-1]*batch_size

            # The last iteration (if we run many steps) repeat the search once.
            if self.repeat == True and outer_step == self.BINARY_SEARCH_STEPS-1:
                CONST = upper_bound

            # set the variables so that we don't have to send them over again
            self.sess.run(self.setup, {self.assign_timg: batch,
                                       self.assign_tlab: batchlab,
                                       self.assign_const: CONST})
            
            prev = np.inf
            for iteration in range(self.MAX_ITERATIONS):
                # perform the attack 
                _, l, l2s, scores, nimg = self.sess.run([self.train, self.loss, 
                                                         self.l2dist, self.output, 
                                                         self.newimg])

                if np.all(scores>=-.0001) and np.all(scores <= 1.0001):
                    if np.allclose(np.sum(scores,axis=1), 1.0, atol=1e-3):
                        if not self.I_KNOW_WHAT_I_AM_DOING_AND_WANT_TO_OVERRIDE_THE_PRESOFTMAX_CHECK:
                            raise Exception("The output of model.predict should return the pre-softmax layer. It looks like you are returning the probability vector (post-softmax). If you are sure you want to do that, set attack.I_KNOW_WHAT_I_AM_DOING_AND_WANT_TO_OVERRIDE_THE_PRESOFTMAX_CHECK = True")
                
                # print out the losses every 10%
                if iteration%(self.MAX_ITERATIONS//10) == 0:
                    print(iteration,self.sess.run((self.loss,self.loss1,self.loss2)))

                # check if we should abort search if we're getting nowhere.
                if self.ABORT_EARLY and iteration%(self.MAX_ITERATIONS//10) == 0:
                    if l > prev*.9999:
                        break
                    prev = l

                # adjust the best result found so far
                for e,(l2,sc,ii) in enumerate(zip(l2s,scores,nimg)):
                    if l2 < bestl2[e] and compare(sc, np.argmax(batchlab[e])):
                        bestl2[e] = l2
                        bestscore[e] = np.argmax(sc)
                    if l2 < o_bestl2[e] and compare(sc, np.argmax(batchlab[e])):
                        o_bestl2[e] = l2
                        o_bestscore[e] = np.argmax(sc)
                        o_bestattack[e] = ii

            # adjust the constant as needed
            for e in range(batch_size):
                if compare(bestscore[e], np.argmax(batchlab[e])) and bestscore[e] != -1:
                    # success, divide const by two
                    upper_bound[e] = min(upper_bound[e],CONST[e])
                    if upper_bound[e] < 1e9:
                        CONST[e] = (lower_bound[e] + upper_bound[e])/2
                else:
                    # failure, either multiply by 10 if no solution found yet
                    #          or do binary search with the known upper bound
                    lower_bound[e] = max(lower_bound[e],CONST[e])
                    if upper_bound[e] < 1e9:
                        CONST[e] = (lower_bound[e] + upper_bound[e])/2
                    else:
                        CONST[e] *= 10

        # return the best solution found
        o_bestl2 = np.array(o_bestl2)
        return o_bestattack


Digit Recognition Model

To ensure the run time was effective on my local machine, I halved the training and test samples. My local colab notebook could not handle the large amount of data, even after the reduction to one epoch. With these variables standard, my logical hyper parameter evaluation will still take place.

In [8]:
## setup_mnist.py -- mnist data and model loading code
##
## Copyright (C) 2016, Nicholas Carlini <nicholas@carlini.com>.
##
## This program is licenced under the BSD 2-Clause licence,
## contained in the LICENCE file in this directory.

import tensorflow as tf
import numpy as np
import os
import pickle
import gzip
import urllib.request

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.models import load_model

def extract_data(filename, num_images):
    with gzip.open(filename) as bytestream:
        bytestream.read(16)
        buf = bytestream.read(num_images*28*28)
        data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
        data = (data / 255) - 0.5
        data = data.reshape(num_images, 28, 28, 1)
        return data

def extract_labels(filename, num_images):
    with gzip.open(filename) as bytestream:
        bytestream.read(8)
        buf = bytestream.read(1 * num_images)
        labels = np.frombuffer(buf, dtype=np.uint8)
    return (np.arange(10) == labels[:, None]).astype(np.float32)

class MNIST:
    def __init__(self):
        if not os.path.exists("data"):
            os.mkdir("data")
            files = ["train-images-idx3-ubyte.gz",
                     "t10k-images-idx3-ubyte.gz",
                     "train-labels-idx1-ubyte.gz",
                     "t10k-labels-idx1-ubyte.gz"]
            for name in files:

                urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/' + name, "data/"+name)

        train_data = extract_data("data/train-images-idx3-ubyte.gz", 30000)
        train_labels = extract_labels("data/train-labels-idx1-ubyte.gz", 30000)
        self.test_data = extract_data("data/t10k-images-idx3-ubyte.gz", 10000)
        self.test_labels = extract_labels("data/t10k-labels-idx1-ubyte.gz", 10000)
        
        VALIDATION_SIZE = 2500
        
        self.validation_data = train_data[:VALIDATION_SIZE, :, :, :]
        self.validation_labels = train_labels[:VALIDATION_SIZE]
        self.train_data = train_data[VALIDATION_SIZE:, :, :, :]
        self.train_labels = train_labels[VALIDATION_SIZE:]


class MNISTModel:
    def __init__(self, restore, session=None):
        self.num_channels = 1
        self.image_size = 28
        self.num_labels = 10

        model = Sequential()

        model.add(Conv2D(32, (3, 3),
                         input_shape=(28, 28, 1)))
        model.add(Activation('relu'))
        model.add(Conv2D(32, (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        model.add(Conv2D(64, (3, 3)))
        model.add(Activation('relu'))
        model.add(Conv2D(64, (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        model.add(Flatten())
        model.add(Dense(200))
        model.add(Activation('relu'))
        model.add(Dense(200))
        model.add(Activation('relu'))
        model.add(Dense(10))
        model.load_weights(restore)

        self.model = model

    def predict(self, data):
        return self.model(data)


Training the neural network to prevent the imminent attack. The results will be less accurate, however these will be assessed on an ad hoc basis when compared to the original testing done by the authors. This is because of the number of epochs. I had to reduce the epochs to 1 so that the program could run. However, this will be the only statically reduced variable. All other variables will be assessed with hyperparameter tuning. We will use the results of epoch=1 as the baseline. If other parameters in addition to the epoch change prove more accurate, I will call into question the researcher's results.

In [9]:
## train_models.py -- train the neural network models for attacking
##
## Copyright (C) 2016, Nicholas Carlini <nicholas@carlini.com>.
##
## This program is licenced under the BSD 2-Clause licence,
## contained in the LICENCE file in this directory.


import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import os


### GLENN's changed
###Assessing model holistically, assessing model's variables

### is structure of the network efficient? Is RELU best activation function

def train(data, file_name, params, num_epochs=1, batch_size=128, train_temp=1, init=None):
    """
    Standard neural network training procedure.
    """
    model = Sequential()

    print(data.train_data.shape)
    
    model.add(Conv2D(params[0], (3, 3),
                            input_shape=data.train_data.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(params[1], (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(params[2], (3, 3)))
    model.add(Activation('relu'))
    model.add(Conv2D(params[3], (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(params[4]))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(params[5]))
    model.add(Activation('relu'))
    model.add(Dense(10))
    
    if init != None:
        model.load_weights(init)

    def fn(correct, predicted):
        return tf.nn.softmax_cross_entropy_with_logits(labels=correct,
                                                       logits=predicted/train_temp)

    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    
    model.compile(loss=fn,
                  optimizer=sgd,
                  metrics=['accuracy'])
    
    model.fit(data.train_data, data.train_labels,
              batch_size=batch_size,
              validation_data=(data.validation_data, data.validation_labels),
              epochs=num_epochs,
              shuffle=True)
    

    if file_name != None:
        model.save(file_name)

    return model



### assessing how the distillation variables work
### manipulating training temperature and epochs and batch size
def train_distillation(data, file_name, params, num_epochs=1, batch_size=128, train_temp=1):
    """
    Train a network using defensive distillation.

    Distillation as a Defense to Adversarial Perturbations against Deep Neural Networks
    Nicolas Papernot, Patrick McDaniel, Xi Wu, Somesh Jha, Ananthram Swami
    IEEE S&P, 2016.
    """
    if not os.path.exists(file_name+"_init"):
        # Train for one epoch to get a good starting point.
        train(data, file_name+"_init", params, 1, batch_size)
    
    # now train the teacher at the given temperature
    teacher = train(data, file_name+"_teacher", params, num_epochs, batch_size, train_temp,
                    init=file_name+"_init")

    # evaluate the labels at temperature t
    predicted = teacher.predict(data.train_data)
    with tf.Session() as sess:
        y = sess.run(tf.nn.softmax(predicted/train_temp))
        print(y)
        data.train_labels = y

    # train the student model at temperature t
    student = train(data, file_name, params, num_epochs, batch_size, train_temp,
                    init=file_name+"_init")

    # and finally we predict at temperature 1
    predicted = student.predict(data.train_data)

    print(predicted)
    
if not os.path.isdir('models'):
    os.makedirs('models')

train(MNIST(), "models/mnist", [32, 32, 64, 64, 200, 200], num_epochs=1)



### Reduced epoch to One---Otherwise machine would not tolerate
train_distillation(MNIST(), "models/mnist-distilled-100", [32, 32, 64, 64, 200, 200],
                   num_epochs=1, train_temp=100)



(27500, 28, 28, 1)


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Train on 27500 samples, validate on 2500 samples
27500/27500 [==============================] - ETA: 0s - loss: 1.1922 - acc: 0.5853

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/engine/training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


27500/27500 [==============================] - 21s 766us/sample - loss: 1.1922 - acc: 0.5853 - val_loss: 0.1738 - val_acc: 0.9488
(27500, 28, 28, 1)


...

Train on 27500 samples, validate on 2500 samples
27500/27500 [==============================] - 20s 741us/sample - loss: 0.8800 - acc: 0.8719 - val_loss: 0.2138 - val_acc: 0.9340


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


[[9.97243524e-01 2.04873718e-08 7.06829887e-05 ... 2.37824196e-07
  1.41560777e-05 5.87734030e-06]
 [4.49844738e-05 1.06764222e-04 2.13190669e-05 ... 1.55094946e-02
  2.54720692e-02 9.31318998e-01]
 [4.91138089e-05 1.48634715e-02 4.75054145e-01 ... 4.66128200e-01
  3.26630403e-03 1.05894124e-03]
 ...
 [5.03622694e-04 4.24358404e-05 3.03298002e-03 ... 3.50739310e-06
  2.18446730e-04 1.05412029e-04]
 [2.45355437e-07 8.59983800e-07 5.41622876e-06 ... 9.98070896e-01
  1.15561363e-06 1.81332312e-03]
 [2.20502116e-05 9.80815947e-01 3.33803683e-03 ... 3.20217805e-03
  6.09757937e-03 5.19232417e-04]]
(27500, 28, 28, 1)


2022-11-18 15:02:18.189969: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open models/mnist-distilled-100_init: DATA_LOSS: not an sstable (bad magic number): perhaps your file is in a different file format and you need to use a different restore operator?


Train on 27500 samples, validate on 2500 samples
27500/27500 [==============================] - 21s 749us/sample - loss: 0.9063 - acc: 0.8992 - val_loss: 0.2677 - val_acc: 0.9208
[[1122.8302   -732.50977   148.45581  ... -381.2876    -39.38256
  -192.51917 ]
 [-377.06287  -229.26973  -499.10898  ...  237.02077   286.85422
   630.6206  ]
 [-375.77194   179.04689   493.96124  ...  442.238      30.352488
  -111.80072 ]
 ...
 [  40.551907 -181.92093   285.29266  ... -437.79102   -12.819769
   -83.18041 ]
 [-341.6484   -287.85733  -223.31908  ... 1074.602    -196.71526
   514.00244 ]
 [-430.3153    617.8414    120.23373  ...   44.937046  167.24411
  -121.69469 ]]


In [10]:
## verify.py -- check the accuracy of a neural network
##
## Copyright (C) 2016, Nicholas Carlini <nicholas@carlini.com>.
##
## This program is licenced under the BSD 2-Clause licence,
## contained in the LICENCE file in this directory.


import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

BATCH_SIZE = 1

with tf.Session() as sess:
    data, model = MNIST(), MNISTModel("models/mnist", sess)

    x = tf.placeholder(tf.float32, (None, model.image_size, model.image_size, model.num_channels))
    y = model.predict(x)

    r = []
    for i in range(0,len(data.test_data),BATCH_SIZE):
        pred = sess.run(y, {x: data.test_data[i:i+BATCH_SIZE]})
        #print(pred)
        #print('real',data.test_labels[i],'pred',np.argmax(pred))
        r.append(np.argmax(pred,1) == np.argmax(data.test_labels[i:i+BATCH_SIZE],1))
        print(np.mean(r))


2022-11-18 15:02:51.329311: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open models/mnist: DATA_LOSS: not an sstable (bad magic number): perhaps your file is in a different file format and you need to use a different restore operator?


1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9705882352941176
0.9714285714285714
0.9722222222222222
0.972972972972973
0.9736842105263158
0.9743589743589743
0.975
0.975609756097561
0.9761904761904762
0.9767441860465116
0.9772727272727273
0.9777777777777777
0.9782608695652174
0.9787234042553191
0.9791666666666666
0.9795918367346939
0.98
0.9803921568627451
0.9807692307692307
0.9811320754716981
0.9814814814814815
0.9818181818181818
0.9821428571428571
0.9824561403508771
0.9827586206896551
0.9830508474576272
0.9833333333333333
0.9836065573770492
0.9838709677419355
0.9841269841269841
0.984375
0.9846153846153847
0.9848484848484849
0.9850746268656716
0.9852941176470589
0.9855072463768116
0.9857142857142858
0.9859154929577465
0.9861111111111112
0.9863013698630136
0.972972972972973
0.9733333333333334
0.9736842105263158
0.974025974025974
0.9743589743589743
0.9746835443037974
0.975
0.9753086419753086
0.97560975

In [27]:



#####################################################


        ## setup_mnist.py -- mnist data and model loading code
##
## Copyright (C) 2016, Nicholas Carlini <nicholas@carlini.com>.
##
## This program is licenced under the BSD 2-Clause licence,
## contained in the LICENCE file in this directory.

import tensorflow as tf
import numpy as np
import os
import pickle
import gzip
import urllib.request

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.models import load_model

def extract_data(filename, num_images):
    with gzip.open(filename) as bytestream:
        bytestream.read(16)
        buf = bytestream.read(num_images*28*28)
        data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
        data = (data / 255) - 0.5
        data = data.reshape(num_images, 28, 28, 1)
        return data

def extract_labels(filename, num_images):
    with gzip.open(filename) as bytestream:
        bytestream.read(8)
        buf = bytestream.read(1 * num_images)
        labels = np.frombuffer(buf, dtype=np.uint8)
    return (np.arange(10) == labels[:, None]).astype(np.float32)

class MNIST:
    def __init__(self):
        if not os.path.exists("data"):
            os.mkdir("data")
            files = ["train-images-idx3-ubyte.gz",
                     "t10k-images-idx3-ubyte.gz",
                     "train-labels-idx1-ubyte.gz",
                     "t10k-labels-idx1-ubyte.gz"]
            for name in files:

                urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/' + name, "data/"+name)

        train_data = extract_data("data/train-images-idx3-ubyte.gz", 30000)
        train_labels = extract_labels("data/train-labels-idx1-ubyte.gz", 30000)
        self.test_data = extract_data("data/t10k-images-idx3-ubyte.gz", 10000)
        self.test_labels = extract_labels("data/t10k-labels-idx1-ubyte.gz", 10000)
        
        VALIDATION_SIZE = 2500
        
        self.validation_data = train_data[:VALIDATION_SIZE, :, :, :]
        self.validation_labels = train_labels[:VALIDATION_SIZE]
        self.train_data = train_data[VALIDATION_SIZE:, :, :, :]
        self.train_labels = train_labels[VALIDATION_SIZE:]


class MNISTModel:
    def __init__(self, restore, session=None):
        self.num_channels = 1
        self.image_size = 28
        self.num_labels = 10

        model = Sequential()

        model.add(Conv2D(32, (3, 3),
                         input_shape=(28, 28, 1)))
        model.add(Activation('relu'))
        model.add(Conv2D(32, (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        model.add(Conv2D(64, (3, 3)))
        model.add(Activation('relu'))
        model.add(Conv2D(64, (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        model.add(Flatten())
        model.add(Dense(200))
        model.add(Activation('relu'))
        model.add(Dense(200))
        model.add(Activation('relu'))
        model.add(Dense(10))
        model.load_weights(restore)

        self.model = model

    def predict(self, data):
        return self.model(data)



###################################################################################

## train_models.py -- train the neural network models for attacking
##
## Copyright (C) 2016, Nicholas Carlini <nicholas@carlini.com>.
##
## This program is licenced under the BSD 2-Clause licence,
## contained in the LICENCE file in this directory.


import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import os


### GLENN's changed
###Assessing model holistically, assessing model's variables

### is structure of the network efficient? Is RELU best activation function

def train(data, file_name, params, num_epochs=1, batch_size=128, train_temp=1, init=None):
    """
    Standard neural network training procedure.
    """
    model = Sequential()

    print(data.train_data.shape)
    
    model.add(Conv2D(params[0], (3, 3),
                            input_shape=data.train_data.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(params[1], (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(params[2], (3, 3)))
    model.add(Activation('relu'))
    model.add(Conv2D(params[3], (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(params[4]))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(params[5]))
    model.add(Activation('relu'))
    model.add(Dense(10))
    
    if init != None:
        model.load_weights(init)

    def fn(correct, predicted):
        return tf.nn.softmax_cross_entropy_with_logits(labels=correct,
                                                       logits=predicted/train_temp)

    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    
    model.compile(loss=fn,
                  optimizer=sgd,
                  metrics=['accuracy'])
    
    model.fit(data.train_data, data.train_labels,
              batch_size=batch_size,
              validation_data=(data.validation_data, data.validation_labels),
              epochs=num_epochs,
              shuffle=True)
    

    if file_name != None:
        model.save(file_name)

    return model



### assessing how the distillation variables work
### manipulating training temperature and epochs and batch size
def train_distillation(data, file_name, params, num_epochs=1, batch_size=128, train_temp=1):
    """
    Train a network using defensive distillation.

    Distillation as a Defense to Adversarial Perturbations against Deep Neural Networks
    Nicolas Papernot, Patrick McDaniel, Xi Wu, Somesh Jha, Ananthram Swami
    IEEE S&P, 2016.
    """
    if not os.path.exists(file_name+"_init"):
        # Train for one epoch to get a good starting point.
        train(data, file_name+"_init", params, 1, batch_size)
    
    # now train the teacher at the given temperature
    teacher = train(data, file_name+"_teacher", params, num_epochs, batch_size, train_temp,
                    init=file_name+"_init")

    # evaluate the labels at temperature t
    predicted = teacher.predict(data.train_data)
    with tf.Session() as sess:
        y = sess.run(tf.nn.softmax(predicted/train_temp))
        print(y)
        data.train_labels = y

    # train the student model at temperature t
    student = train(data, file_name, params, num_epochs, batch_size, train_temp,
                    init=file_name+"_init")

    # and finally we predict at temperature 1
    predicted = student.predict(data.train_data)

    print(predicted)
    
if not os.path.isdir('models'):
    os.makedirs('models')

train(MNIST(), "models/mnist", [32, 32, 64, 64, 200, 200], num_epochs=1)



### Reduced epoch to One---Otherwise machine would not tolerate
train_distillation(MNIST(), "models/mnist-distilled-100", [32, 32, 64, 64, 200, 200],
                   num_epochs=1, train_temp=100)



#################################################################################


## verify.py -- check the accuracy of a neural network
##
## Copyright (C) 2016, Nicholas Carlini <nicholas@carlini.com>.
##
## This program is licenced under the BSD 2-Clause licence,
## contained in the LICENCE file in this directory.


import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

BATCH_SIZE = 1

with tf.Session() as sess:
    data, model = MNIST(), MNISTModel("models/mnist", sess)

    x = tf.placeholder(tf.float32, (None, model.image_size, model.image_size, model.num_channels))
    y = model.predict(x)

    r = []
    for i in range(0,len(data.test_data),BATCH_SIZE):
        pred = sess.run(y, {x: data.test_data[i:i+BATCH_SIZE]})
        #print(pred)
        #print('real',data.test_labels[i],'pred',np.argmax(pred))
        r.append(np.argmax(pred,1) == np.argmax(data.test_labels[i:i+BATCH_SIZE],1))
        print(np.mean(r))



#############################################

## l2_attack.py -- attack a network optimizing for l_2 distance
##
## Copyright (C) 2016, Nicholas Carlini <nicholas@carlini.com>.
##
## This program is licenced under the BSD 2-Clause licence,
## contained in the LICENCE file in this directory.

import sys
import tensorflow as tf
import numpy as np

binary_steps = [1,2,3,4,5,6,7,8,9]
max_iterations = [1000,2000,5000,10000,15000]
learn_rate=[1e-2,1e-1,1e-3,1e-4,1e-5,1,2,0.5]
confidence = [1,2,3,0]         # how strong the adversarial example should be
INITIAL_CONST = 1e-3     # the initial constant c to pick as a first guess


###After testing the boolean variable, targeted == false, Glenn determined that the score was more accurate when it was not targeted. 
### Then, Glenn moved on to manipulate the different integer and float variables using hyperparameter tuning
distortion = -1.5
while distortion > distortion*1.5:
    ABORT_EARLY = True       # if we stop improving, abort gradient descent early
    TARGETED = False          # should we target one specific class? or just be wrong?

    for i in range(len(binary_steps)):

        BINARY_SEARCH_STEPS = binary_steps[i]
        for j in range(len(max_iterations)): # number of times to adjust the constant with binary search
            MAX_ITERATIONS = max_iterations[j]
            for x in range(len(learn_rate)):
                LEARNING_RATE = learn_rate[x]     # larger values converge faster to less accurate results
                for y in range(len(confidence)):
                    CONFIDENCE = confidence[y]           # how strong the adversarial example should be

    ### GLENN's changed
    ###Assessing model holistically, assessing model's variables

    ### Will be using for loop to see if these are indeed the best variables for the job

                    class CarliniL2:
                        def __init__(self, sess, model, batch_size=1, confidence = CONFIDENCE,
                                    targeted = TARGETED, learning_rate = LEARNING_RATE,
                                    binary_search_steps = BINARY_SEARCH_STEPS, max_iterations = MAX_ITERATIONS,
                                    abort_early = ABORT_EARLY, 
                                    initial_const = INITIAL_CONST,
                                    boxmin = -0.5, boxmax = 0.5):
                            """
                            The L_2 optimized attack. 

                            This attack is the most efficient and should be used as the primary 
                            attack to evaluate potential defenses.

                            Returns adversarial examples for the supplied model.

                            confidence: Confidence of adversarial examples: higher produces examples
                            that are farther away, but more strongly classified as adversarial.
                            batch_size: Number of attacks to run simultaneously.
                            targeted: True if we should perform a targetted attack, False otherwise.
                            learning_rate: The learning rate for the attack algorithm. Smaller values
                            produce better results but are slower to converge.
                            binary_search_steps: The number of times we perform binary search to
                            find the optimal tradeoff-constant between distance and confidence. 
                            max_iterations: The maximum number of iterations. Larger values are more
                            accurate; setting too small will require a large learning rate and will
                            produce poor results.
                            abort_early: If true, allows early aborts if gradient descent gets stuck.
                            initial_const: The initial tradeoff-constant to use to tune the relative
                            importance of distance and confidence. If binary_search_steps is large,
                            the initial constant is not important.
                            boxmin: Minimum pixel value (default -0.5).
                            boxmax: Maximum pixel value (default 0.5).
                            """

                            image_size, num_channels, num_labels = model.image_size, model.num_channels, model.num_labels
                            self.sess = sess
                            self.TARGETED = targeted
                            self.LEARNING_RATE = learning_rate
                            self.MAX_ITERATIONS = max_iterations
                            self.BINARY_SEARCH_STEPS = binary_search_steps
                            self.ABORT_EARLY = abort_early
                            self.CONFIDENCE = confidence
                            self.initial_const = initial_const
                            self.batch_size = batch_size

                            self.repeat = binary_search_steps >= 10

                            self.I_KNOW_WHAT_I_AM_DOING_AND_WANT_TO_OVERRIDE_THE_PRESOFTMAX_CHECK = False

                            shape = (batch_size,image_size,image_size,num_channels)
                            
                            # the variable we're going to optimize over
                            modifier = tf.Variable(np.zeros(shape,dtype=np.float32))

                            # these are variables to be more efficient in sending data to tf
                            self.timg = tf.Variable(np.zeros(shape), dtype=tf.float32)
                            self.tlab = tf.Variable(np.zeros((batch_size,num_labels)), dtype=tf.float32)
                            self.const = tf.Variable(np.zeros(batch_size), dtype=tf.float32)

                            # and here's what we use to assign them
                            self.assign_timg = tf.placeholder(tf.float32, shape)
                            self.assign_tlab = tf.placeholder(tf.float32, (batch_size,num_labels))
                            self.assign_const = tf.placeholder(tf.float32, [batch_size])
                            
                            # the resulting image, tanh'd to keep bounded from boxmin to boxmax
                            self.boxmul = (boxmax - boxmin) / 2.
                            self.boxplus = (boxmin + boxmax) / 2.
                            self.newimg = tf.tanh(modifier + self.timg) * self.boxmul + self.boxplus
                            
                            # prediction BEFORE-SOFTMAX of the model
                            self.output = model.predict(self.newimg)
                            
                            # distance to the input data
                            self.l2dist = tf.reduce_sum(tf.square(self.newimg-(tf.tanh(self.timg) * self.boxmul + self.boxplus)),[1,2,3])
                            
                            # compute the probability of the label class versus the maximum other
                            real = tf.reduce_sum((self.tlab)*self.output,1)
                            other = tf.reduce_max((1-self.tlab)*self.output - (self.tlab*10000),1)

                            if self.TARGETED:
                                # if targetted, optimize for making the other class most likely
                                loss1 = tf.maximum(0.0, other-real+self.CONFIDENCE)
                            else:
                                # if untargeted, optimize for making this class least likely.
                                loss1 = tf.maximum(0.0, real-other+self.CONFIDENCE)

                            # sum up the losses
                            self.loss2 = tf.reduce_sum(self.l2dist)
                            self.loss1 = tf.reduce_sum(self.const*loss1)
                            self.loss = self.loss1+self.loss2
                            
                            # Setup the adam optimizer and keep track of variables we're creating
                            start_vars = set(x.name for x in tf.global_variables())
                            optimizer = tf.train.AdamOptimizer(self.LEARNING_RATE)
                            self.train = optimizer.minimize(self.loss, var_list=[modifier])
                            end_vars = tf.global_variables()
                            new_vars = [x for x in end_vars if x.name not in start_vars]

                            # these are the variables to initialize when we run
                            self.setup = []
                            self.setup.append(self.timg.assign(self.assign_timg))
                            self.setup.append(self.tlab.assign(self.assign_tlab))
                            self.setup.append(self.const.assign(self.assign_const))
                            
                            self.init = tf.variables_initializer(var_list=[modifier]+new_vars)

                        def attack(self, imgs, targets):
                            """
                            Perform the L_2 attack on the given images for the given targets.

                            If self.targeted is true, then the targets represents the target labels.
                            If self.targeted is false, then targets are the original class labels.
                            """
                            r = []
                            print('go up to',len(imgs))
                            for i in range(0,len(imgs),self.batch_size):
                                print('tick',i)
                                r.extend(self.attack_batch(imgs[i:i+self.batch_size], targets[i:i+self.batch_size]))
                            return np.array(r)

                        def attack_batch(self, imgs, labs):
                            """
                            Run the attack on a batch of images and labels.
                            """
                            def compare(x,y):
                                if not isinstance(x, (float, int, np.int64)):
                                    x = np.copy(x)
                                    if self.TARGETED:
                                        x[y] -= self.CONFIDENCE
                                    else:
                                        x[y] += self.CONFIDENCE
                                    x = np.argmax(x)
                                if self.TARGETED:
                                    return x == y
                                else:
                                    return x != y

                            batch_size = self.batch_size

                            # convert to tanh-space
                            imgs = np.arctanh((imgs - self.boxplus) / self.boxmul * 0.999999)

                            # set the lower and upper bounds accordingly
                            lower_bound = np.zeros(batch_size)
                            CONST = np.ones(batch_size)*self.initial_const
                            upper_bound = np.ones(batch_size)*1e10

                            # the best l2, score, and image attack
                            o_bestl2 = [1e10]*batch_size
                            o_bestscore = [-1]*batch_size
                            o_bestattack = [np.zeros(imgs[0].shape)]*batch_size
                            
                            for outer_step in range(self.BINARY_SEARCH_STEPS):
                                print(o_bestl2)
                                # completely reset adam's internal state.
                                self.sess.run(self.init)
                                batch = imgs[:batch_size]
                                batchlab = labs[:batch_size]
                        
                                bestl2 = [1e10]*batch_size
                                bestscore = [-1]*batch_size

                                # The last iteration (if we run many steps) repeat the search once.
                                if self.repeat == True and outer_step == self.BINARY_SEARCH_STEPS-1:
                                    CONST = upper_bound

                                # set the variables so that we don't have to send them over again
                                self.sess.run(self.setup, {self.assign_timg: batch,
                                                        self.assign_tlab: batchlab,
                                                        self.assign_const: CONST})
                                
                                prev = np.inf
                                for iteration in range(self.MAX_ITERATIONS):
                                    # perform the attack 
                                    _, l, l2s, scores, nimg = self.sess.run([self.train, self.loss, 
                                                                            self.l2dist, self.output, 
                                                                            self.newimg])

                                    if np.all(scores>=-.0001) and np.all(scores <= 1.0001):
                                        if np.allclose(np.sum(scores,axis=1), 1.0, atol=1e-3):
                                            if not self.I_KNOW_WHAT_I_AM_DOING_AND_WANT_TO_OVERRIDE_THE_PRESOFTMAX_CHECK:
                                                raise Exception("The output of model.predict should return the pre-softmax layer. It looks like you are returning the probability vector (post-softmax). If you are sure you want to do that, set attack.I_KNOW_WHAT_I_AM_DOING_AND_WANT_TO_OVERRIDE_THE_PRESOFTMAX_CHECK = True")
                                    
                                    # print out the losses every 10%
                                    if iteration%(self.MAX_ITERATIONS//10) == 0:
                                        print(iteration,self.sess.run((self.loss,self.loss1,self.loss2)))

                                    # check if we should abort search if we're getting nowhere.
                                    if self.ABORT_EARLY and iteration%(self.MAX_ITERATIONS//10) == 0:
                                        if l > prev*.9999:
                                            break
                                        prev = l

                                    # adjust the best result found so far
                                    for e,(l2,sc,ii) in enumerate(zip(l2s,scores,nimg)):
                                        if l2 < bestl2[e] and compare(sc, np.argmax(batchlab[e])):
                                            bestl2[e] = l2
                                            bestscore[e] = np.argmax(sc)
                                        if l2 < o_bestl2[e] and compare(sc, np.argmax(batchlab[e])):
                                            o_bestl2[e] = l2
                                            o_bestscore[e] = np.argmax(sc)
                                            o_bestattack[e] = ii

                                # adjust the constant as needed
                                for e in range(batch_size):
                                    if compare(bestscore[e], np.argmax(batchlab[e])) and bestscore[e] != -1:
                                        # success, divide const by two
                                        upper_bound[e] = min(upper_bound[e],CONST[e])
                                        if upper_bound[e] < 1e9:
                                            CONST[e] = (lower_bound[e] + upper_bound[e])/2
                                    else:
                                        # failure, either multiply by 10 if no solution found yet
                                        #          or do binary search with the known upper bound
                                        lower_bound[e] = max(lower_bound[e],CONST[e])
                                        if upper_bound[e] < 1e9:
                                            CONST[e] = (lower_bound[e] + upper_bound[e])/2
                                        else:
                                            CONST[e] *= 10

                            # return the best solution found
                            o_bestl2 = np.array(o_bestl2)
                            print(o_bestl2)
                            return o_bestattack



                    ######################################################################
                    ## test_attack.py -- sample code to test attack procedure
                    ##
                    ## Copyright (C) 2016, Nicholas Carlini <nicholas@carlini.com>.
                    ##
                    ## This program is licenced under the BSD 2-Clause licence,
                    ## contained in the LICENCE file in this directory.


                    import tensorflow.compat.v1 as tf
                    tf.disable_v2_behavior()
                    import numpy as np
                    import time
                    import random



                    def show(img):
                        """
                        Show MNSIT digits in the console.
                        """
                        remap = "  .*#"+"#"*100
                        img = (img.flatten()+.5)*3
                        if len(img) != 784: return
                        print("START")
                        for i in range(28):
                            print("".join([remap[int(round(x))] for x in img[i*28:i*28+28]]))


                    def generate_data(data, samples, targeted=True, start=0, inception=False):
                        """
                        Generate the input data to the attack algorithm.

                        data: the images to attack
                        samples: number of samples to use
                        targeted: if true, construct targeted attacks, otherwise untargeted attacks
                        start: offset into data to use
                        inception: if targeted and inception, randomly sample 100 targets intead of 1000
                        """
                        inputs = []
                        targets = []
                        for i in range(samples):
                            if targeted:
                                if inception:
                                    seq = random.sample(range(1,1001), 10)
                                else:
                                    seq = range(data.test_labels.shape[1])

                                for j in seq:
                                    if (j == np.argmax(data.test_labels[start+i])) and (inception == False):
                                        continue
                                    inputs.append(data.test_data[start+i])
                                    targets.append(np.eye(data.test_labels.shape[1])[j])
                            else:
                                inputs.append(data.test_data[start+i])
                                targets.append(data.test_labels[start+i])

                        inputs = np.array(inputs)
                        targets = np.array(targets)

                        return inputs, targets


                    if __name__ == "__main__":
                        with tf.Session() as sess:
                            data, model =  MNIST(), MNISTModel("models/mnist", sess)
                            attack = CarliniL2(sess, model, batch_size=9, max_iterations=1000, confidence=0)

                            inputs, targets = generate_data(data, samples=1, targeted=True,
                                                            start=0, inception=False)
                            timestart = time.time()
                            adv = attack.attack(inputs, targets)
                            timeend = time.time()
                            
                            print("Took",timeend-timestart,"seconds to run",len(inputs),"samples.")

                            for i in range(len(adv)):
                                print("Valid:")
                                show(inputs[i])
                                print("Adversarial:")
                                show(adv[i])
                                
                                print("Classification:", model.model.predict(adv[i:i+1]))
                                distortion=np.sum((adv[i]-inputs[i])**2)**.5
                                print("Total distortion:", np.sum((adv[i]-inputs[i])**2)**.5)

                    print(distortion)



(27500, 28, 28, 1)
Train on 27500 samples, validate on 2500 samples
27500/27500 [==============================] - 22s 810us/sample - loss: 1.0940 - acc: 0.6352 - val_loss: 0.1964 - val_acc: 0.9416
(27500, 28, 28, 1)


2022-11-18 16:16:17.497482: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open models/mnist-distilled-100_init: DATA_LOSS: not an sstable (bad magic number): perhaps your file is in a different file format and you need to use a different restore operator?


Train on 27500 samples, validate on 2500 samples
27500/27500 [==============================] - 22s 785us/sample - loss: 0.8826 - acc: 0.8712 - val_loss: 0.2148 - val_acc: 0.9392
[[9.98557866e-01 2.97965332e-08 5.49059878e-05 ... 1.81771455e-07
  6.93484208e-06 7.49749370e-06]
 [1.08121676e-04 2.71302502e-04 1.03916260e-04 ... 7.28375139e-03
  1.31483406e-01 8.43644917e-01]
 [5.26177355e-05 1.57140810e-02 8.05239141e-01 ... 6.80742562e-02
  5.91542386e-03 3.81594989e-04]
 ...
 [2.61362788e-04 4.02758560e-05 1.80146331e-03 ... 1.03405341e-06
  2.18479676e-04 7.51227417e-05]
 [2.44574085e-06 6.34073285e-06 4.21795958e-05 ... 9.96663034e-01
  1.34261991e-05 2.24593631e-03]
 [5.19670248e-05 9.57888603e-01 1.04929581e-02 ... 4.38001845e-03
  1.54747749e-02 7.83830124e-04]]
(27500, 28, 28, 1)


2022-11-18 16:16:50.626197: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open models/mnist-distilled-100_init: DATA_LOSS: not an sstable (bad magic number): perhaps your file is in a different file format and you need to use a different restore operator?


Train on 27500 samples, validate on 2500 samples
27500/27500 [==============================] - 22s 815us/sample - loss: 0.9164 - acc: 0.8979 - val_loss: 0.2655 - val_acc: 0.9204
[[1193.5879   -768.4932     89.815506 ... -477.98035   -57.738907
  -198.15005 ]
 [-339.6869   -282.8302   -415.1914   ...  134.92268   363.50107
   549.74817 ]
 [-327.00873   134.20479   605.88574  ...  364.11203    46.524483
  -165.37372 ]
 ...
 [  57.097702 -195.05481   210.8955   ... -521.7389     38.432568
   -70.29572 ]
 [-291.58176  -307.89508   -92.73201  ... 1001.8154   -154.38734
   406.28918 ]
 [-393.6264    567.231     149.11497  ...    9.542887  159.17755
  -139.67094 ]]


...

2022-11-18 16:19:53.314147: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open models/mnist: DATA_LOSS: not an sstable (bad magic number): perhaps your file is in a different file format and you need to use a different restore operator?


go up to 9
tick 0
[10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0]
0 (0.0, 0.0, 0.0)
100 (0.0, 0.0, 0.0)
200 (0.0, 0.0, 0.0)
300 (0.0, 0.0, 0.0)
400 (0.0, 0.0, 0.0)
500 (0.0, 0.0, 0.0)
600 (0.0, 0.0, 0.0)
700 (0.0, 0.0, 0.0)
800 (0.0, 0.0, 0.0)
900 (0.0, 0.0, 0.0)
[0. 0. 0. 0. 0. 0. 0. 0. 0.]
Took 10.366426944732666 seconds to run 9 samples.
Valid:
START
                            
                            
                            
                            
                            
                            
                            
       ...                  
      ******.........       
          .**********.      
                   **       
                   *.       
                  **        
                 .**        
                 **         
                .*.         
                .*          
                *.          
               **           
     

2022-11-18 16:20:07.320456: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open models/mnist: DATA_LOSS: not an sstable (bad magic number): perhaps your file is in a different file format and you need to use a different restore operator?


go up to 9
tick 0
[10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0]
0 (0.0, 0.0, 0.0)
100 (0.0, 0.0, 0.0)


KeyboardInterrupt: 